Извлечение мнений из новостных текстов

[Репозиторий на GitHub ](https://github.com/dialogue-evaluation/RuOpinionNE-2024)

[Страница на CodaLab](https://codalab.lisn.upsaclay.fr/competitions/20244)

## Данные

### Загрузка

Каждый объект датасета представляет собой json-строку. Напишем функцию для загрузки данных.

In [ ]:
import json
import requests

def load_json(path, url):

  r = requests.get(url, allow_redirects=True)
  with open(path, 'wb') as st:
    st.write(r.content)

  with open(path, 'r', encoding = 'utf8') as file:
    data = list()
    for line in file:
        data_entry = json.loads(line)
        data.append(data_entry)
  return data

Загрузим данные обучающей, валидационной и тестовой выборки. Все части, кроме тестовой, содержат разметку мнений.

In [ ]:
path = "train.jsonl"
url = 'https://raw.githubusercontent.com/dialogue-evaluation/RuOpinionNE-2024/master/train.jsonl'
train_data = load_json(path, url)
print(f"Количество предложений обучающей выборки: {len(train_data)}\n")
train_data[16]

In [ ]:
path = "validation.jsonl"
url = 'https://raw.githubusercontent.com/dialogue-evaluation/RuOpinionNE-2024/master/validation_labeled.jsonl'
validation_data = load_json(path, url)
print(f"Количество предложений валидационной выборки: {len(validation_data)}\n")
validation_data[400]

In [ ]:
path = "test.jsonl"
url = 'https://raw.githubusercontent.com/dialogue-evaluation/RuOpinionNE-2024/master/test.jsonl'
test_data = load_json(path, url)
print(f"Количество предложений тестовой выборки: {len(test_data)}\n")
test_data[763]

### Анализ

Проанализиуем обучающую выборку.

Определим минимальную, максимальную и среднюю длину текста. Отобразим распределение на графике.

In [ ]:
lens = [len(x['text'].split()) for x in train_data]

max_l, min_l, mean_l = max(lens), min(lens), sum(lens)/len(lens)

print(f'Минимальная длина текста: {min_l}')
print(f'Максимальная длина текста: {max_l}')
print(f'Средняя длина текста: {mean_l:.3f}')

In [ ]:
from collections import Counter
from matplotlib import pyplot as plt

len_counts = Counter(lens)
plt.figure(figsize = (6,3))
plt.bar(len_counts.keys(), len_counts.values())

Выведем самый длинный текст.

In [ ]:
for elem in train_data:
    if len(elem['text'].split()) == max_l:
        print(elem['sent_id'])
        print(elem['text'])

Определим количество текстов, состоящих из более чем одного предложения:

In [ ]:
count = 0
for elem in train_data:
    text = elem['text']
    if '\n' in text or '.' in text[:-1]:
        count+=1
print(count)

Для дальнейшего анализа преобразуем обучающую выборку в датафрейм, где 1 мнение = 1 строка.

In [ ]:
opinions = []
for elem in train_data:
    for op in elem['opinions']:
        # источник и интервал
        if len(op['Source'][0]) > 1:
            source_text = op['Source'][0]
            source_span = op['Source'][1]
        else:
            source_text = op['Source'][0][0]
            source_span = op['Source'][1][0]
        # объект и интервал
        if len(op['Target'][0]) > 1:
            target_text = op['Target'][0]
            target_span = op['Target'][1]
        else:
            target_text = op['Target'][0][0]
            target_span = op['Target'][1][0]
        # выражение и интервал
        if len(op['Polar_expression'][0]) > 1:
            exp_text = op['Polar_expression'][0]
            exp_span = op['Polar_expression'][1]
        else:
            exp_text = op['Polar_expression'][0][0]
            exp_span = op['Polar_expression'][1][0]
        opinions.append([elem['sent_id'], elem['text'], source_text, target_text, exp_text,
                       op['Polarity'], source_span, target_span, exp_span])

In [ ]:
import pandas as pd
cols = ['sent_id', 'text', 'Source', 'Target', 'Polar_expression', 'Polarity', 'Source_span', 'Target_span', 'Polar_expression_span']
df = pd.DataFrame(opinions, columns = cols)
df

Оценим распределение классов тональности.

In [ ]:
print(df['Polarity'].value_counts())

plt.figure(figsize = (6,3))
df['Polarity'].value_counts().plot.bar()

Определим число текстов с множественными мнениями и отобразим распределение на графике.

In [ ]:
textcount = Counter(df['text'].value_counts())
textcount

In [ ]:
plt.figure(figsize = (6,3))
plt.bar(textcount.keys(), textcount.values())

Посчитаем максимальную и среднюю длину для источника, объекта и выражения. Выведем примеры.

In [ ]:
sources = []
for item in df['Source']:
    if isinstance(item, str):
        sources.append(item)
    else:
        sources+=item
max_s = max([len(x.split()) for x in sources])
mean_s = sum([len(x.split()) for x in sources])/len(sources)
print(f'Максимальная длина источника: {max_s}')
print(f'Средняя длина источника: {mean_s:.3f}')

print(f'Самый длиный источник:')
for item in df['Source']:
    if isinstance(item, str) and len(item.split()) == max_s:
        print(f'"{item}"')

In [ ]:
targets = []
for item in df['Target']:
    if isinstance(item, str):
        targets.append(item)
    else:
        targets+=item
max_t = max([len(x.split()) for x in targets])
mean_t = sum([len(x.split()) for x in targets])/len(targets)
print(f'Максимальная длина объекта: {max_t}')
print(f'Средняя длина объекта: {mean_t:.3f}')

for item in df['Target']:
    if isinstance(item, str) and len(item.split()) == max_t:
        print(f'Самый длиный объект:\n"{item}"')

In [ ]:
expressions = list()
for item in df['Polar_expression']:
    if isinstance(item, str):
        expressions.append(item)
    else:
        expressions+=item
max_e = max([len(x.split()) for x in expressions])
mean_e = sum([len(x.split()) for x in expressions])/len(expressions)
print(f'Максимальная длина выражения: {max_e}')
print(f'Средняя длина выражения: {mean_e:.3f}')

for item in df['Polar_expression']:
    if isinstance(item, str) and len(item.split()) == max_e:
        print(f'Самое длиное выражение:\n"{item}"')

Определим самые частые источники.

In [ ]:
df['Source'].value_counts()

Посчитаем количество разрывных и множественных источников, объектов и выражений.

In [ ]:
s, t, e = 0, 0, 0
for i, row in df.iterrows():
    if isinstance(row['Source'], list):
        s+=1
    if isinstance(row['Target'], list):
        t+=1
    if isinstance(row['Polar_expression'], list):
        e+=1

print(f'Множественный/ фрагментированный источник: {s}')
print(f'Множественный/ фрагментированный объект: {t}')
print(f'Множественное/ фрагментированное выражение: {e}')

Выведем примеры.

In [ ]:
for i, row in df.iterrows():
    if isinstance(row['Polar_expression'], list):
        print(row['Polar_expression'], row['Polar_expression_span'], sep = '\n')

## Модель

Применим модель [Qwen2.5 72B instruct](https://huggingface.co/Qwen/Qwen2.5-72B-Instruct) в режиме few-shot.

### InferenceClient

 Воспользуемся моделью через Hugging Face API.

1. Регистрируемся на [Hugging Face](https://huggingface.co/).
2. Создаем токен в настройках аккаунта: Settings -> [Access Tokens](https://huggingface.co/settings/tokens). Важно: выбираем тип токена `read`.
3. Записываем токен в переменную TOKEN

Не храните токен на GitHub! Если вы хотите распространить ваш код, предварительно удалите токен из кода.

In [ ]:
token = 'hf_00000' # токен начинается с hf_...
model_name = "Qwen/Qwen2.5-72B-Instruct"

3. Импортируем [InferenceClient](https://huggingface.co/docs/huggingface_hub/v0.16.2/en/package_reference/inference_client#huggingface_hub.InferenceClient) — инструмент для получения запросов от модели.
4. Указываем в клиенте название модели и токен, которые мы прописали заранее.

In [ ]:
from huggingface_hub import InferenceClient

client = InferenceClient(model_name, token=token)

Создаем промт. Можно задать следующие параметры:

* messages — системная роль и промпт для LLM
* max_tokens — максимальная длина вывода (в токенах)
* temperature — температура (рандомность выдачи)
* top_p — также задает рандомность, а именно количество вариантов вывода модели на каждом шаге генерации выдачи

Что записывается в messages:

* системная роль: основной промпт модели, например, ты помощник преподавателя или генерируй код на Python
* текущий промпт: можно получать его через input, переменную или аргумент функции и т.д.

Рассмотрим пример промта, который можно было бы использовать при создании виртуального ассистента.

In [ ]:
topic = input('Введите тему запроса: ') # здесь мы просим пользователя ввести перменную

output = client.chat.completions.create( # метод из HuggingFace Cient для осуществления запросов к LLM
          messages=[
              {"role": "system", # маркер системной роли
                "content": "Ты ассистент обучающегося в университете. Объясняй концепты. Используй формат маркированных списков."
              },
              {"role": "user", # маркер текущего промпта пользователя
              "content": f"Объясни основы {topic} простыми словами"},
          ],
          max_tokens=1000, # мы задали максимальную длину ответа — 1000 токенов, это значение можно увеличить или уменьшить
          temperature=0.2, # temperature можно поменять, например, на 0.6 или 0.7 — понаблюдайте, окажет ли это влияние на результат
          top_p=0.9 # это значение можно задавать от 0.1 до 0.9 — также можете понаблюдать за изменениями
          ).choices[0].get('message')['content'] # этот хвостик нам нужен, чтобы вывести только ответ модели без метаданных — попробуйте удалить его и вывести ответ с метаданными
print(output)

### Применение к данным

Опишем промт для задачи извлечения мнений.

In [ ]:
prompt = 'Твоя задача состоит в том, чтобы проанализировать текст и извлечь из него выражения мнений, \
представленные в виде кортежа мнений, состоящих из 4 основных составляющих:\n\
1. Источник мнения: автор, именованная сущность текста (подстрока исходного текста), либо "NULL". Key = Source;\n\
2. Объект мнения: именованная сущность в тексте (подстрока исходного текста). Key = Target;\n\
3. Тональность: положительная/негативная ("POS"/ "NEG"). Допустимы только значения "POS" и "NEG", значение "NULL" недопустимо. Key = Polarity;\n\
4. Языковое выражение: аргумент, на основании которого принята результирующая тональность \
(одна или несколько подстрок исходного текста). Key = Polar_expression;\n\
Значение источника, объекта и тональности должны быть заключены в кавычки. \
Если источник мнения отсутствует, то Source = "NULL". Если источником мнения является автор, то Source = "AUTHOR". Источник мнения не может быть выражен местоимением. \
В прочих случаях поле Source должно полностью совпадать с подстрокой исходного текста. Поля Target, Polar_expression всегда полностью совпадают с подстроками текста и стоять в том же падеже.\n\
Не добавляй никаких пояснений. Ответ необходимо представить в виде json списка, каждый элемент которого является кортежем мнений. Само слово json не нужно выводить. \
Каждый кортеж мнений это словарь, состоящий из четырех значений: Source, Target, Polarity, Polar_expression. \
Для извлечённых Source, Target, Polarity, Polar_expression должно быть справедливо утверждение: \
На основании выражения Polar_expression можно сказать, что Source имеет Polarity отношение к Target..\n\
Ниже представлены примеры выполнения задачи:\n\
***Текст***\n\
Премьер-министр Молдовы осудил террориста за бесчеловечные и жестокие действия.\n\
Source: "Премьер-министр Молдовы", Target: "террориста", Polarity: "NEG", Polar_expression: "бесчеловечные и жестокие действия".\n\
***Текст***\n\
Знаменитая актриса продемонстрировала человечность и простоту, достойную уважения публики.\n\
***Ответ***\n\
Source: "AUTHOR", Target: "актриса", Polarity: "POS", Polar_expression: "продемонстрировала человечность и простоту, достойную уважения публики".\n\
Проанализируй таким же образом следующий текст.\n\
***Текст***'

Напишем функцию, которая будет принимать предложения из датасета, записывать его в промт и подавать на вход модели.

In [ ]:
def inference(sentence):
  output = client.chat.completions.create(
          messages=[
              {"role": "user",
              "content": f"{prompt}\n{sentence}"},
          ],
          stream=False,
          temperature=0.5,
          top_p=0.9
          ).choices[0].get('message')['content']
  return output

Применим функцию к одному из предложений.

In [ ]:
sent_id = 0
text = train_data[sent_id]["text"]
print(f'Prompt:\n{prompt}\n{text}\n')
output = inference(train_data[sent_id]["text"])
print(f'Predicted tuple: {type(output)}\n{output}\n')
print(f'Gold tuple: {type(train_data[sent_id]["opinions"])}\n{train_data[sent_id]["opinions"]}')

LLM не возвращает интервалы для источника, объекта и выражения. Также отличается формат истинной цепочки мнений и предсказания. Следовательно, необходима некоторая дополнительная обработка ответов модели.

Для примера применим функцию к первым 100 предложениям обучающей выборки.

In [ ]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(100))
train_output = []

for sample in train_data[:100]:
  sample_output = inference(sample["text"])
  train_output.append(sample_output)
  progress_bar.update(1)

Определим функцию для нахождения интервалов и функцию для преобразования ответа модели в формат json-строки с нужными ключами "`sent_id`", "`text`" и "`opinions`".

In [ ]:
def get_interval(text, phrase):
    if phrase == 'AUTHOR':
        return 'NULL'
    elif phrase == 'NULL':
        return '0:0'
    else:
        start_index = text.find(phrase)
        end_index = start_index + len(phrase)
        return f"{start_index}:{end_index}"

In [ ]:
def string2json(sample, output):

    # Определяем идентификатор
    sent_id = sample["sent_id"]
    # Определяем предложение
    text = sample["text"]

    if output == '[]':
        predicted_json = {
            "sent_id": sent_id,
            "text": text,
            "opinions": []
            }

    else:
        # Преобразуем строку в Python-объект (список)
        opinions_list = json.loads(output)
        if opinions_list[0]['Polarity']=='NULL':
          predicted_json = {
            "sent_id": sent_id,
            "text": text,
            "opinions": []
            }
        else:
          # Создаем новый JSON-объект
          predicted_json = {
              "sent_id": sent_id,
              "text": text,
              "opinions": [{"Source": [[opinions_list[0]["Source"]], [get_interval(text, opinions_list[0]["Source"])]],
                            "Target": [[opinions_list[0]["Target"]], [get_interval(text, opinions_list[0]["Target"])]],
                            "Polar_expression": [[opinions_list[0]["Polar_expression"]], [get_interval(text, opinions_list[0]["Polar_expression"])]],
                            "Polarity": opinions_list[0]["Polarity"]}]
              }
    return predicted_json

Применяем обработку ко всем ответам модели.

In [ ]:
train_json_output = []
for i in range(len(train_output)):
  print(i, string2json(train_data[i], train_output[i]))
  train_json_output.append(string2json(train_data[i], train_output[i]))

In [ ]:
train_json_output[0]

In [ ]:
train_data[0]

Для оценки качества нужно посчитать метрику $Sentiment\;Tuple\;F_1$.

Импортируем код, предлагаемый организаторами соревнования.

In [ ]:
!wget -q https://raw.githubusercontent.com/dialogue-evaluation/RuOpinionNE-2024/master/codalab/evaluation.py

In [ ]:
import evaluation

In [ ]:
evaluation.do_eval_core(train_data[:100], train_json_output)